In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import os
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Add, ReLU
from tensorflow.keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.backend import log, epsilon, mean
from tensorflow.keras.applications import ResNet152


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
csv_file_path = '/kaggle/input/flood-data/Kaggle/devset_images_gt.csv'
df = pd.read_csv(csv_file_path)


image_folder_path = '/kaggle/input/flood-data/Kaggle/devset_images/devset_images'
image_files = [file for file in os.listdir(image_folder_path) if not file.startswith('._') and file.endswith('.jpg')]


selected_image_files = []
selected_labels = []

for image_file in image_files:
    image_id = int(image_file.split('.')[0]) 
    label = df[df['id'] == image_id]['label'].values
    if len(label) > 0:
        selected_image_files.append(image_file)
        selected_labels.append(label[0])


X = []
y = []

for image_file, label in zip(selected_image_files, selected_labels):
    image_path = os.path.join(image_folder_path, image_file)
    image = Image.open(image_path)
    image = image.resize((256, 256)) 
    image = np.array(image) 
    X.append(image)
    y.append(label)

X = np.array(X)
y = np.array(y)


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [3]:
# from keras.preprocessing.image import ImageDataGenerator

# # Create an instance of ImageDataGenerator with desired augmentations
# datagen = ImageDataGenerator(
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

In [9]:

epochs = 50
batch_size = 32


history = model.fit(
    X_train, y_train,
    batch_size=batch_size,
    steps_per_epoch=len(X_train) // batch_size,
    epochs=epochs,
    validation_data=(X_test, y_test)
)


Epoch 1/50
148/148 [==============================] - 45s 200ms/step - loss: 3.8588 - accuracy: 0.8190 - val_loss: 0.3019 - val_accuracy: 0.8691
Epoch 2/50
148/148 [==============================] - 26s 175ms/step - loss: 0.2621 - accuracy: 0.9003 - val_loss: 0.3249 - val_accuracy: 0.8767
Epoch 3/50
148/148 [==============================] - 26s 173ms/step - loss: 0.1258 - accuracy: 0.9458 - val_loss: 0.4813 - val_accuracy: 0.8767
Epoch 4/50
148/148 [==============================] - 26s 173ms/step - loss: 0.0742 - accuracy: 0.9669 - val_loss: 0.5705 - val_accuracy: 0.8653
Epoch 5/50
148/148 [==============================] - 26s 174ms/step - loss: 0.0392 - accuracy: 0.9813 - val_loss: 0.4681 - val_accuracy: 0.8767
Epoch 6/50
148/148 [==============================] - 26s 175ms/step - loss: 0.0280 - accuracy: 0.9883 - val_loss: 0.5578 - val_accuracy: 0.8861
Epoch 7/50
148/148 [==============================] - 26s 174ms/step - loss: 0.0106 - accuracy: 0.9945 - val_loss: 0.7075 - val_ac

In [10]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Loss on test set:", loss)
print("Accuracy on test set:", accuracy)

17/17 [==============================] - 3s 153ms/step - loss: 1.6781 - accuracy: 0.8937
Loss on test set: 1.6781070232391357
Accuracy on test set: 0.8937381505966187


In [11]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image

test_image_folder_path = '/kaggle/input/flood-data/Kaggle/testset_images/testset_images'
test_image_files = [file for file in os.listdir(test_image_folder_path) if not file.startswith('._')]

X_test = []

for image_file in test_image_files:
    image_id_str = image_file.split('.')[0]
    image_id = int(image_id_str)
    image_path = os.path.join(test_image_folder_path, image_file)
    image = Image.open(image_path)
    image = image.resize((256, 256))
    image = np.array(image)
    X_test.append(image)

X_test = np.array(X_test)


test_predictions = model.predict(X_test)


test_df = pd.DataFrame({'id': [int(file.split('.')[0]) for file in test_image_files],
                        'label': (test_predictions > 0.5).astype(int).flatten()})

test_predictions_csv_path = '/kaggle/working/test_predictions.csv'
test_df.to_csv(test_predictions_csv_path, index=False)

print(f'Test predictions saved to {test_predictions_csv_path}')

42/42 [==============================] - 10s 159ms/step
Test predictions saved to /kaggle/working/test_predictions.csv


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152 (Functional)      (None, 8, 8, 2048)        58370944  
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                                 
 dense (Dense)               (None, 256)               33554688  
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 91,925,889
Trainable params: 33,554,945
Non-trainable params: 58,370,944
_________________________________________________________________
